In [27]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from geopandas.tools import geocode
from folium import Marker

In [9]:
uni = pd.read_csv('top_universities.csv')
uni.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [8]:
UG ='University of Ghana'
SA = 'University of Pretoria'
MA = 'Makerere Univeristy'
NG = 'University of Ibadan'
KE = 'University of Nairobi'

In [11]:
top_uni = uni.append(pd.DataFrame([UG,SA,MA,NG,KE], columns=uni.columns), ignore_index=True)
top_uni.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [12]:
def coder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Lat':point.y, 'Long':point.x,'geometry':point})
    except:
        None
top_uni[['Latitude','Longitude','Geometry']]=top_uni.apply(lambda x: coder(x['Name']), axis=1)
top_uni.head()

,Name,Latitude,Longitude,Geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.255668482609204 51.75870755)
1,University of Cambridge,52.199852,0.119739,POINT (0.1197386574107438 52.1998523)
2,Imperial College London,51.498871,-0.175608,POINT (-0.175607955839404 51.49887085)
3,ETH Zurich,47.377327,8.547509,POINT (8.5475089 47.3773269)
4,UCL,51.524126,-0.132930,POINT (-0.1329302373595478 51.52412645)


In [15]:
#what percentage of universities were successfully loaded
print("{}% of universities were loaded".format(
    (1-sum(np.isnan(top_uni['Latitude']))/len(top_uni))*100)
)

90.47619047619048% of universities were loaded


In [23]:
#Drop all unloaded universities
top_uni=top_uni.loc[~np.isnan(top_uni['Latitude'])]
top_uni.head()

,Name,Latitude,Longitude,Geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.255668482609204 51.75870755)
1,University of Cambridge,52.199852,0.119739,POINT (0.1197386574107438 52.1998523)
2,Imperial College London,51.498871,-0.175608,POINT (-0.175607955839404 51.49887085)
3,ETH Zurich,47.377327,8.547509,POINT (8.5475089 47.3773269)
4,UCL,51.524126,-0.132930,POINT (-0.1329302373595478 51.52412645)


In [25]:
#convert to GeoDataFrame
univ = gpd.GeoDataFrame(top_uni, geometry=top_uni.Geometry)
univ.crs={'init':'4326'}
univ.head()

,Name,Latitude,Longitude,Geometry,geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.255668482609204 51.75870755),POINT (-1.25567 51.75871)
1,University of Cambridge,52.199852,0.119739,POINT (0.1197386574107438 52.1998523),POINT (0.11974 52.19985)
2,Imperial College London,51.498871,-0.175608,POINT (-0.175607955839404 51.49887085),POINT (-0.17561 51.49887)
3,ETH Zurich,47.377327,8.547509,POINT (8.5475089 47.3773269),POINT (8.54751 47.37733)
4,UCL,51.524126,-0.132930,POINT (-0.1329302373595478 51.52412645),POINT (-0.13293 51.52413)


In [30]:
#Map Europe
map=folium.Map(location=[54,15], tiles='cartodbpositron',zoom_start=5)

#Add universities
for idx, row in univ.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(map)
map